# Ayudantía: Regex y Web Services
Javier Dreves y Benjamin Earle

La meta de esta ayudantía es obtener y modificar tweets de un usuario de twitter.

Para lograr esto se deberá conseguir el usuario las palabras que deberán ser modificadas de los _tweets_.Estas palabras se encuentran alteradas y pueden obtenerlas [en este link](https://demo5989374.mockable.io). Para hacer esto, deberán utilizar REgex y una serie de reglas para limpiar el texto.

Una vez obtenidas las palabras a modificar, se deberán obtener los _tweets_ del usuario utilizando la API de Twitter. Luego, se deberán retweetear con las modificaciones pedidas.

### Parte1: Obtención de las palabras a modificar

Las palabras se encuentran [aquí](https://demo5989374.mockable.io).

Las reglas que debemos utilizar para obtener el texto original son:

- Cada vez que se aparece el string `'<Dos Puntos>'` se debe reemplazar con `':'`.
- Cada vez que aparece un `'$'` seguido de uno o más números, este se debe reemplazar con un string vacío.
- Cada vez que aparece el string `'NO ESPACIO'` o `'NO ESPACIOS'` también se debe reemplazar con un string vacío.
- Cada vez que aparece `'=='` seguido de una o más letras minúsculas y seguido denuevo por `'=='` también se reemplaza con un string vacío (Ejemplo: `'==inxqo=='`).
- Por último, para obtener un documento JSON válido, es necesario reemplazar los `'` con `"`.

Recuerden que para ir revisando sus patrones ReGex pueden ingresar [aquí](https://regex101.com/). 

### Limpiador de instrucciones (Con RegEx)

In [ ]:
import re
def limpiador(txt:str):
    """
    txt: String a limpiar
    return: String limpio
    Limpia el string de acuerdo a los patrones definidos. Guarda el resultado para poder verlo.
    """
    patterns = ['<Dos Puntos>', '\$[0-9]+', 'NO_ESPACIO(S)?', '={2}[a-z]+={2}']
    txt = re.sub(patterns.pop(0), ':', txt)
    for pat in patterns:
        txt = re.sub(pat, '', txt)
    txt = re.sub("'", '"', txt)
    return txt

### Uso de api para obtener las instrucciones

In [ ]:
import json
import requests
def obtener_instrucciones():
    URL = "https://demo5989374.mockable.io/"
    # Api con el usuario y las frases a reemplazar en sus tweets
    my_api = requests.get(URL)
    # Desencriptar el json obtenido en la api
    json_string = limpiador(my_api.json())
    json_dict = json.loads(json_string)
    return json_dict

In [ ]:
instrucciones = obtener_instrucciones()
print(instrucciones)

Como pueden ver, las instrucciones tienen 2 elementos, el nombre de usuario y las frases a reemplazar.

In [ ]:
usuario = instrucciones['user']
frases = instrucciones['phrases_to_replace']

### Parte2: API de Twitter

En esta parte deberán obtener los _tweets_ del usuario y luego retweetaerlos con su cuenta que pueden obtener [aquí](https://developer.twitter.com/content/developer-twitter/en.html). 

OJO: Twitter es bien quisquilloso por lo que es necesario validar la cuenta con las keys (Para esto se usa la librería `requests_oauthlib`).

In [ ]:
from requests_oauthlib import OAuth1
# Aqui deben ingresar sus credenciales
API_KEY = ""
API_SECRET = ""
ACCESS_TOKEN = ""
ACCESS_TOKEN_SECRET = ""
auth = OAuth1(API_KEY, API_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET) # Necesario para twitter

In [ ]:
# Se obtienen los 100 primeros tweets del usuario
tweets_request = requests.get("https://api.twitter.com/1.1/statuses/user_timeline.json",
                 params={"screen_name": usuario, "count": "100", "tweet_mode":"extended"}, auth=auth)

tweets = (tweet["full_text"] for tweet in tweets_request.json()) # Lista de los tweets del usuario

In [ ]:
for tweet in tweets:
    t = [] # Contiene los matches de este tweet
    for a_reemplazar, txt_reemplazo in frases.items():
        t.extend([re.search(a_reemplazar.lower(), tweet),
                  re.search(a_reemplazar.capitalize(), tweet),
                  re.search(a_reemplazar, tweet)])

        txt_reemplazo = "\"" + txt_reemplazo + "\"" # Agregar comilla al reemplazo
    

        # Reemplazos
        new_tweet = re.sub(a_reemplazar.lower(), txt_reemplazo, tweet)
        new_tweet = re.sub(a_reemplazar.capitalize(), txt_reemplazo, tweet)
        new_tweet = re.sub(a_reemplazar, txt_reemplazo, tweet)
        new_tweet = re.sub('@', '', tweet) # Para evitar spamear las menciones

    if any(t): # Imprimir los que hicieron algun match
        print("Match: ", *filter(bool, t))
        print("Tweet original: ", tweet) # Tweet original
        print("Tweet modificado: ", new_tweet,) # Tweet modificado
        to_tweet = input('>>>>>>>>>>>>>>> Quieres twittearlo? (Y/N) -> ')
        if to_tweet == 'Y':
            requests.post('https://api.twitter.com/1.1/statuses/update.json?',
                         params={"status": new_tweet}, auth=auth)